In [2]:
%load_ext autoreload
%autoreload 2
%cd /group/transreg/sathi/DeepDifE 

import pickle
import importlib
import esparto
import optuna
import numpy as np
import pandas as pd
from evoaug_tf import evoaug, augment
from src.diff_expression_model import get_model, get_siamese_model, post_hoc_conjoining, get_auroc
from skopt.utils import use_named_args

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/storage/nas6/group/biocomp/projects/transreg/sathi/DeepDifE


## Prepare the data

For now we start from this data pickle as I'm not aware how Helder did DE analysis and generated the labels

In [3]:
ppath = "/group/transreg/heopd/dlpipe/results/ath/aba/dlresults/predetermined_dataset/dataset_solid_chrome.pkl"
with open(ppath, 'rb') as f:
    data = pickle.load(f)

In [4]:
data.columns

Index(['Category', 'GeneFamily', 'seqs', 'ohs', 'rcohs', 'ohsDuo',
       'in_original_balanced', 'set', 'npshap-single', 'npshap-posthoc'],
      dtype='object')

In [5]:
data["set"].value_counts()

train    1900
valid     257
test      241
Name: set, dtype: int64

To show how to subdivide the dataset into train-test split we only take the following columns

In [6]:
dataset = data.reset_index()
dataset = dataset[["geneID", "Category", "GeneFamily", "seqs"]]
dataset.rename(columns={"geneID":"GeneID", "Category":"Label", "seqs": "Sequence"}, inplace=True)

In [7]:
dataset

,GeneID,Label,GeneFamily,Sequence
0,AT4G27120,0,HOM04D000881,TAGAGAAGACAAGCGGTTATTTCGTAATTTCCCAGCGACTTTGAAA...
1,AT4G19600,0,HOM04D000740,GTCAAGTAGTGAAATCAAGGTGTGAAGTAAGCTGAGGACAGATAAT...
2,AT3G60880,0,HOM04D003119,AGTTGATATTGAATGAAATCTTCATGTTTTTTGATAAATGATTATA...
3,AT5G06960,0,HOM04D000319,CACTTGTCAGATTCTTCTTACCAAATCCATCAACAAATAAGCAAAT...
4,AT1G14890,0,HOM04D000273,TTGATATAACAGATTCAACACTAAAAATGAGTAAAATCTAAAAAAG...
...,...,...,...,...
2393,AT5G64230,1,HOM04D003278,AAGAAAGAAAAACCGTACATAAACACCCATCTGGTATACCATCGTC...
2394,AT5G64780,1,HOM04D002552,TTTTAGAAAGAAGAAGAAGGATTATTGCCTTATTGGTGAAGGGAAG...
2395,AT4G30470,1,HOM04D000082,TATGTACAGTCTCTACATTTTTTCAAATACATTTTTTTCTTTTTCA...
2396,AT3G51895,1,HOM04D000270,TGGTAAATAATTAAATATATAAGAACATTATTCTAAAGCGTTGAAT...


### One-hot-encode & reverse-complement

In [8]:
from src.prepare_dataset import one_hot_encode_series, reverse_complement_series, reverse_complement_sequence
dataset["One_hot_encoded"] = one_hot_encode_series(dataset["Sequence"])

In [9]:
dataset["RC_one_hot_encoded"] = reverse_complement_series(dataset["One_hot_encoded"])

In [10]:
dataset

,GeneID,Label,GeneFamily,Sequence,One_hot_encoded,RC_one_hot_encoded
0,AT4G27120,0,HOM04D000881,TAGAGAAGACAAGCGGTTATTTCGTAATTTCCCAGCGACTTTGAAA...,"[[0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 1, 0], [1,...","[[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0,..."
1,AT4G19600,0,HOM04D000740,GTCAAGTAGTGAAATCAAGGTGTGAAGTAAGCTGAGGACAGATAAT...,"[[0, 0, 1, 0], [0, 0, 0, 1], [0, 1, 0, 0], [1,...","[[0, 0, 1, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0,..."
2,AT3G60880,0,HOM04D003119,AGTTGATATTGAATGAAATCTTCATGTTTTTTGATAAATGATTATA...,"[[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [0,...","[[0, 0, 0, 1], [0, 0, 1, 0], [1, 0, 0, 0], [0,..."
3,AT5G06960,0,HOM04D000319,CACTTGTCAGATTCTTCTTACCAAATCCATCAACAAATAAGCAAAT...,"[[0, 1, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0,...","[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [1,..."
4,AT1G14890,0,HOM04D000273,TTGATATAACAGATTCAACACTAAAAATGAGTAAAATCTAAAAAAG...,"[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 1, 0], [1,...","[[0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 0, 1], [1,..."
...,...,...,...,...,...,...
2393,AT5G64230,1,HOM04D003278,AAGAAAGAAAAACCGTACATAAACACCCATCTGGTATACCATCGTC...,"[[1, 0, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1,...","[[0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1], [0,..."
2394,AT5G64780,1,HOM04D002552,TTTTAGAAAGAAGAAGAAGGATTATTGCCTTATTGGTGAAGGGAAG...,"[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0,...","[[0, 0, 0, 1], [0, 0, 1, 0], [1, 0, 0, 0], [0,..."
2395,AT4G30470,1,HOM04D000082,TATGTACAGTCTCTACATTTTTTCAAATACATTTTTTTCTTTTTCA...,"[[0, 0, 0, 1], [1, 0, 0, 0], [0, 0, 0, 1], [0,...","[[1, 0, 0, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0,..."
2396,AT3G51895,1,HOM04D000270,TGGTAAATAATTAAATATATAAGAACATTATTCTAAAGCGTTGAAT...,"[[0, 0, 0, 1], [0, 0, 1, 0], [0, 0, 1, 0], [0,...","[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 1, 0], [1,..."


### Train-test split

In [11]:
from src.prepare_dataset import grouped_shuffle_split
train_df, test_df = grouped_shuffle_split(dataset, dataset["GeneFamily"], 0.2)

In [12]:
print(f"Length of training set: {train_df.shape[0]}")
print(f"Length of test set: {test_df.shape[0]}")

Length of training set: 1900
Length of test set: 498


## Cross validation

We first initialize the object that are static throughout the cross validation

In [15]:
augment_list = [
    augment.RandomRC(rc_prob=0.5),
    augment.RandomInsertionBatch(insert_min=0, insert_max=20),
    augment.RandomDeletion(delete_min=0, delete_max=30),
    augment.RandomTranslocationBatch(shift_min=0, shift_max=20),
    augment.RandomMutation(mutate_frac=0.05),
    augment.RandomNoise()
]

2024-07-09 12:57:05.056212: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /software/shared/apps/x86_64/dependencies_rl9:/software/shared/apps/x86_64/git/2.13.1/lib64/
2024-07-09 12:57:05.056294: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-07-09 12:57:05.056369: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cyclone4.psblocal): /proc/driver/nvidia/version does not exist


In [18]:
# early stopping callback
import tensorflow as tf

early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
											patience=20,
											verbose=1,
											mode='min',
											restore_best_weights=True)
# reduce learning rate callback
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
												factor=0.1,
												patience=5,
												min_lr=1e-7,
												mode='min',
												verbose=1)
callbacks = [early_stopping_callback, reduce_lr]

In [16]:
input_shape = train_df["One_hot_encoded"].iloc[0].shape

We prepare the data

In [13]:
def get_input_and_labels(df):
	ohe_np = np.stack(df["One_hot_encoded"])
	rc_np = np.stack(df["RC_one_hot_encoded"])

	x = np.append(ohe_np, rc_np, axis=0)
	x = x.astype('float32')
	y = np.append(df["Label"], df["Label"])
	return x, y

In [14]:
X, Y = get_input_and_labels(train_df)

We will now create the groups. Because we are using both forward and reverse complement we have to concat them with itself

In [27]:
groups = pd.concat([train_df["GeneFamily"], train_df["GeneFamily"]], axis = 0) 

Initialize the splitter

In [38]:
from sklearn.model_selection import GroupKFold

group_kfold = GroupKFold(n_splits=5)

# Define metric containers
loss_list = []
accuracy_list = []
auroc_list = []
auprc_list = []
true_positive_list = []

for i, (train_index, validation_index) in enumerate(group_kfold.split(X, Y, groups)):
	x_train = X[train_index]
	y_train = Y[train_index]

	x_val = X[validation_index]
	y_val = Y[validation_index]

	model = get_model(input_shape=input_shape, perform_evoaug=True, augment_list=augment_list, learning_rate=0.001)

	# We add validation here, because one of the callbacks relies on val_loss metric
	model.fit(x_train,
			y_train,
			epochs=100,	
			batch_size=100,
			validation_data=(x_val, y_val),
			callbacks=callbacks
			)
	scores = model.evaluate(x_val, y_val, verbose=0)
	print(f'Score for fold {i}: Loss of {scores[0]}; accuracy of {scores[1]}; auroc of {scores[2]}; auprc of {scores[3]}; TP of {scores[4]}')
	
	loss_list.append(scores[0])
	accuracy_list.append(scores[1])
	auroc_list.append(scores[2])
	auprc_list.append(scores[3])
	true_positive_list.append(scores[4])
	

print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(loss_list)):
  print('------------------------------------------------------------------------')
  print(f'Score for fold {i}: Loss of {loss_list[i]}; accuracy of {accuracy_list[i]}; AUROC of {auroc_list[i]}; AUPRC of {auprc_list[i]}; TP of {true_positive_list[i]}')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Loss: {np.mean(loss_list)} (+- {np.std(loss_list)})')
print(f'> Auroc: {np.mean(auroc_list)}')
print('------------------------------------------------------------------------')

31/31 [==============================] - 26s 446ms/step - loss: 0.6955 - acc: 0.4875 - auROC: 0.4793 - auPRC: 0.4706 - true_positives_18: 657.0000 - val_loss: 0.6938 - val_acc: 0.4987 - val_auROC: 0.5060 - val_auPRC: 0.5235 - val_true_positives_18: 1.0000 - lr: 0.0010
Score for fold 0: Loss of 0.6937711238861084; accuracy of 0.4973684251308441; auroc of 0.505862295627594; auprc of 0.5241748094558716; TP of 0.0
31/31 [==============================] - 26s 424ms/step - loss: 0.6947 - acc: 0.5010 - auROC: 0.4933 - auPRC: 0.4809 - true_positives_19: 321.0000 - val_loss: 0.6935 - val_acc: 0.4882 - val_auROC: 0.5405 - val_auPRC: 0.5534 - val_true_positives_19: 22.0000 - lr: 0.0010
Score for fold 1: Loss of 0.6934657692909241; accuracy of 0.4973684251308441; auroc of 0.5365015864372253; auprc of 0.5551764965057373; TP of 27.0
31/31 [==============================] - 27s 439ms/step - loss: 0.6960 - acc: 0.4809 - auROC: 0.4775 - auPRC: 0.4794 - true_positives_20: 876.0000 - val_loss: 0.6934 - v

## Test set

We will train a model on all the training data

In [40]:
model = get_model(input_shape=input_shape, perform_evoaug=True, augment_list=augment_list, learning_rate=0.001)
history = model.fit(X,
					Y,
					epochs=100,
					batch_size=100,
					)

38/38 [==============================] - 25s 341ms/step - loss: 0.6952 - acc: 0.4913 - auROC: 0.4859 - auPRC: 0.4793 - true_positives_23: 784.0000


We perform post-hoc conjoing

In [41]:
siamese_model = get_siamese_model(model.model)

Prepare the test set data

In [42]:
x_test = np.stack(test_df["One_hot_encoded"])
x_test_rc = np.stack(test_df["RC_one_hot_encoded"])

In [43]:
y_test = test_df["Label"].to_numpy()

As we used evo aug in our model all the sequences which were trained on, were of length 620. We use the evoaug padding function to pad the test set to the same length

In [44]:
x_test = model._pad_end(x_test)
x_test_rc = model._pad_end(x_test_rc)

In [45]:
predictions_categories, predictions = post_hoc_conjoining(siamese_model, x_test, x_test_rc)

16/16 [==============================] - 1s 58ms/step


In [46]:
get_auroc(y_test, predictions)

0.5053068609553966

In [47]:
a=1